<b>Most Similar Document Finding by TfIdf</b>

In [1]:
import pandas as pd
from collections import Counter
import nltk
import numpy as np
np.seterr(divide='ignore', invalid='ignore')
import math

def pre_process(p_document):
    words = p_document.split()
    counter_word = Counter(words)        
    dict_word_count = nltk.defaultdict(int)
    for w in counter_word: 
        dict_word_count[w] += 1             
    return dict_word_count;

def tf(p_term, p_document):
    return float(p_document.count(p_term)) / len(p_document)

def idf(p_term, p_total_word_count, p_data):
    article_count = 1
    for indice, row in p_data.iterrows():
        if p_term in row['icerik']:
            article_count += 1
    return math.log(p_total_word_count/float(article_count))

def tfidf_score(p_query_words, p_article, p_total_word_count, p_df_documents):
    result = 0
    for w in p_query_words:
        result += (tf(w, p_article) * idf(w, p_total_word_count, p_df_documents))
    return result
    
def find_similar_document(p_df_documents, p_query_words):
    total_word_count = len(p_query_words)
    for indice, row in p_df_documents.iterrows():
        dict_doc = pre_process(row['icerik'])
        total_word_count += len(dict_doc)

    for indice, row in p_df_documents.iterrows():
        if indice % 1000 == 0:
            print("İşlem yapılan makale numarası: " + str(row['makale_no']))
        p_df_documents.at[indice, 'tfidf_skoru'] = tfidf_score(p_query_words, row['icerik'], total_word_count, p_df_documents)
    
    p_df_documents = p_df_documents.sort_values(by = 'tfidf_skoru', ascending = False)
    print(p_df_documents.head(10)[['sira', 'makale_no', 'tfidf_skoru']])

In [2]:
query = "makine öğrenmesi deep learning expert finding"
query_words = query.split()

In [3]:
df_docs = pd.read_csv(("C://icerik_dergipark.csv"), sep=";", encoding="utf8")
df_docs.loc[:,'tfidf_skoru'] = 0.0
df_docs.head()

,sira,makale_no,icerik,tfidf_skoru
0,1,1,determination of ımportant building constructi...,0.0
1,2,2,damage ıdentification analyses of a historic m...,0.0
2,3,3,an effective ımproved multiobjective evolution...,0.0
3,4,4,structural equation model of the factors affec...,0.0
4,5,5,consolidated undrained monotonic shearing resp...,0.0


In [4]:
df_docs_sobiad = pd.read_csv(("C://icerik_sobiad.csv"), sep=";", encoding="utf8")
df_docs_sobiad.loc[:,'tfidf_skoru'] = 0.0
df_docs_sobiad.head()

,sira,makale_no,icerik,tfidf_skoru
0,1,1,bır parametrelı elıptık homotetık düzlemsel ha...,0.0
1,2,2,total ırregularıty of ındubala product of grap...,0.0
2,3,3,bir boyutlu kesme problemi için üç amaçlı bir ...,0.0
3,4,4,güç kalitesindeki bozulma türlerinin sınıfland...,0.0
4,5,5,bir oluklu mukavva kutu fabrikasında standart ...,0.0


In [ ]:
find_similar_document(df_docs, query_words)
find_similar_document(df_docs_sobiad, query_words)

In [ ]:
df_docs[['sira', 'makale_no', 'tfidf_skoru']].to_csv("C://tfidf_score_dergipark.csv", index=False)
df_docs_sobiad[['sira', 'makale_no', 'tfidf_skoru']].to_csv("C://tfidf_score_sobiad.csv", index=False)